In [1]:
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader("../data/Smart_Job_Internship_Recommendation_Platform__1768917114.pdf")
     


c:\Users\ultra\OneDrive\FOLDER\Semester 3\ML Projects\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

In [2]:
from langchain_community.document_loaders import PyPDFLoader
from pathlib import Path
import os
def documents_procesors(dir_name):
    documents_list=[]
    dir_name_p=Path(dir_name)

    file_names=dir_name_p.glob("**/*.pdf")

    for file_name in file_names:
        loader=PyPDFLoader(file_name)
        documents=loader.load()
        for doc in documents:
            doc.metadata["file_name"]=file_name.name
            doc.metadata["file_type"]="pdf"

        documents_list.extend(documents)
        print(documents)
    return documents_list
      
all_pdf_documents=documents_procesors("../data/")

[Document(metadata={'producer': 'pdfTeX-1.40.27', 'creator': 'LaTeX with hyperref', 'creationdate': '2026-01-15T13:16:22+00:00', 'author': '', 'keywords': '', 'moddate': '2026-01-15T13:16:22+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.27 (TeX Live 2025) kpathsea version 6.4.1', 'subject': '', 'title': '', 'trapped': '/False', 'source': '..\\data\\Smart_Job_Internship_Recommendation_Platform__1768917114.pdf', 'total_pages': 45, 'page': 0, 'page_label': '1', 'file_name': 'Smart_Job_Internship_Recommendation_Platform__1768917114.pdf', 'file_type': 'pdf'}, page_content='Ibn Tofail University - ENSAK\nProgram: Computer Engineering\nProject Report\nJava Application for Internship and Job Search in\nMorocco\nPrepared by:\nRhemimet Meryem\nKhaliqi Salma\nSupervised by:\nProf. Dr. Moumen Aniss\nAcademic Year: 2024 – 2025'), Document(metadata={'producer': 'pdfTeX-1.40.27', 'creator': 'LaTeX with hyperref', 'creationdate': '2026-01-15T13:16:22+00:00', 'author': '', 'k

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

def text_splitter(documents,chunk_size=1000,chunk_overlap=200):
    Text_splitter=RecursiveCharacterTextSplitter(chunk_size= chunk_size,chunk_overlap= chunk_overlap)
    chunks = Text_splitter.split_documents(documents)

    return chunks


chunks= text_splitter(all_pdf_documents)

In [8]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS


class EmbeddingsManager:

   def __init__(self,model_name: str="sentence-transformers/all-MiniLM-L6-v2"):
        self.model_name=model_name
        self.embeddings=None
        self._load_modle()
   
   def _load_modle(self):
     print(f"Loading embedding model: {self.model_name}")
     self.embeddings= HuggingFaceEmbeddings(model_name=self.model_name)
   def get_embeddings(self):
     return self.embeddings


embeddingmanager = EmbeddingsManager()
embeddings = embeddingmanager.get_embeddings()


vectorstore=FAISS.from_documents(chunks,embeddings)
vectorstore.save_local("faiss_index")

Loading embedding model: sentence-transformers/all-MiniLM-L6-v2


C:\Users\ultra\AppData\Local\Temp\ipykernel_13788\4015421888.py:14: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  self.embeddings= HuggingFaceEmbeddings(model_name=self.model_name)
Loading weights: 100%|██████████| 103/103 [00:00<00:00, 193.84it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [12]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import os
from dotenv import load_dotenv
load_dotenv()


retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":4})

llm=ChatGroq(
    model="meta-llama/llama-4-maverick-17b-128e-instruct",      # or "llama3-70b-8192", "mixtral-8x7b-32768"
    temperature=0.1,                   # low = more factual
    max_tokens=500
)


prompt = ChatPromptTemplate.from_template(
    """You are a helpful assistant answering questions based on the provided context.
Use only the following pieces of context to answer the question.
If you don't know or the context doesn't contain the answer, say "I don't have enough information".

Context:
{context}

Question: {question}

Answer concisely:"""
)

rag_chain = (
    {"context": retriever | (lambda docs: "\n\n".join(doc.page_content for doc in docs)),   # format docs
     "question": RunnablePassthrough()}                                                 # pass question through
    | prompt
    | llm
    | StrOutputParser()
)


question = "What is the main idea of this document?"
answer = rag_chain.invoke(question)
print("Answer:", answer)

Answer: The document appears to be a project report or thesis, detailing the implementation of an application, its features, and technologies used.
